In [1]:
import pdfkit
import selenium
import os
import json
import time

In [2]:
# Link to Google Scholar profile
linkScholar = "https://scholar.google.com/citations?user=k09vJggAAAAJ&hl=en"
# Link to Google Chrome downloads folder
download_folder="/Users/giometto/Downloads"
# Time allowed to complete captchas
timeCaptcha = 30

# Using Chrome to access web
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import chromedriver_binary

chrome_options = webdriver.ChromeOptions()
settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--kiosk-printing')

driver = webdriver.Chrome(chrome_options=chrome_options)

# Opens Google Scholar profile
driver.get(linkScholar)

# Clicks SHOW MORE button
search_bar = driver.find_element_by_id("gsc_bpf_more")
search_bar.send_keys(Keys.RETURN)

# Wait for show more to appear
time.sleep(2)

# Changes title which will correspond to file name
driver.execute_script('document.title="{}";'.format('Ch5 00 first page'));
# Saves as pdf
driver.execute_script('window.print();')

# Find citation links (used to see how many papers there are)
citation_links = driver.find_elements_by_xpath('//a[contains(@href, "cites=")]')

for paper in range(0,len(citation_links)):
    # Search for links to papers summaries
    paper_pages = driver.find_elements_by_xpath('//a[contains(@href, "/citations?view_op=view_citation")]')
    # Clicks on next paper summary
    paper_pages[paper].click()

    # Check for captcha
    for i in range(0,2):
        iscaptcha = driver.find_elements_by_xpath('//form[@id="gs_captcha_f"]')
        # 10 seconds to solve captcha
        if len(iscaptcha)>0:
            time.sleep(timeCaptcha)
        iscaptcha = driver.find_elements_by_xpath('//form[@id="captcha-form"]')
        # 10 seconds to solve captcha
        if len(iscaptcha)>0:
            time.sleep(timeCaptcha)

    # Changes title which will correspond to file name
    title = 'Ch5 ' + f"{paper:02}" + ' page 00'
    driver.execute_script('document.title="{}";'.format(title)); 
    driver.execute_script('window.print();')

    # Goes back to Google Scholar profile
    driver.get(linkScholar)

    # Clicks SHOW MORE button
    search_bar = driver.find_element_by_id("gsc_bpf_more")
    search_bar.send_keys(Keys.RETURN)

    # Wait for show more to appear
    time.sleep(2)

    citation_links = driver.find_elements_by_xpath('//a[contains(@href, "cites=")]')
    citation_links[paper].click()

    # Check for captcha
    for i in range(0,2):
        iscaptcha = driver.find_elements_by_xpath('//form[@id="gs_captcha_f"]')
        # 10 seconds to solve captcha
        if len(iscaptcha)>0:
            time.sleep(timeCaptcha)
        iscaptcha = driver.find_elements_by_xpath('//form[@id="captcha-form"]')
        # 10 seconds to solve captcha
        if len(iscaptcha)>0:
            time.sleep(timeCaptcha)
    
    title = 'Ch5 ' + f"{paper:02}" + ' page 01'
    driver.execute_script('document.title="{}";'.format(title)); 
    driver.execute_script('window.print();')

    # Click on next citation page
    citation_pages_next = driver.find_elements_by_xpath('//span[@class="gs_ico gs_ico_nav_next"]')

    page=1
    
    while len(citation_pages_next)>0:
        page = page+1
        citation_pages_next[0].click()
        # Check for captcha
        for i in range(0,2):
            iscaptcha = driver.find_elements_by_xpath('//form[@id="gs_captcha_f"]')
            # 10 seconds to solve captcha
            if len(iscaptcha)>0:
                time.sleep(timeCaptcha)
            iscaptcha = driver.find_elements_by_xpath('//form[@id="captcha-form"]')
            # 10 seconds to solve captcha
            if len(iscaptcha)>0:
                time.sleep(timeCaptcha)

        title = 'Ch5 ' + f"{paper:02}" + ' page ' + f"{page:02}"
        driver.execute_script('document.title="{}";'.format(title)); 
        driver.execute_script('window.print();')
        # Click on next citation page
        citation_pages_next = driver.find_elements_by_xpath('//span[@class="gs_ico gs_ico_nav_next"]')

    # Go back to Google Scholar profile
    driver.get(linkScholar)

    # Clicks SHOW MORE button
    search_bar = driver.find_element_by_id("gsc_bpf_more")
    search_bar.send_keys(Keys.RETURN)

    # Wait for show more to appear
    time.sleep(2)

/var/folders/qj/lp1nfkjx1y515k588c4by_400000gn/T/ipykernel_45470/385809969.py:28: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)
/var/folders/qj/lp1nfkjx1y515k588c4by_400000gn/T/ipykernel_45470/385809969.py:34: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_id("gsc_bpf_more")
/var/folders/qj/lp1nfkjx1y515k588c4by_400000gn/T/ipykernel_45470/385809969.py:46: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  citation_links = driver.find_elements_by_xpath('//a[contains(@href, "cites=")]')
/var/folders/qj/lp1nfkjx1y515k588c4by_400000gn/T/ipykernel_45470/385809969.py:50: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  paper_pages = driver.find_elements_by_xpath('//a[contains(@href, "/citations?view_op=view_citation")]')
/var/folders/q

In [21]:
# Concatenates pdf files generated previously
from PyPDF2 import PdfFileMerger
import os, fnmatch

def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

# find('Ch5*.pdf', '/path/to/Chrome_Downloads')
pdfs = sorted(find('Ch5*.pdf', download_folder))

merger = PdfFileMerger()

for pdf in pdfs:
    merger.append(pdf)

merger.write(download_folder+"/Ch5.pdf")
merger.close()